In [1]:
from wiggler_radiation.wiggler_radiation import get_photon_flux_3D

In [2]:
i3d0 = get_photon_flux_3D()

In [3]:
from scipy.interpolate import RegularGridInterpolator

In [4]:
i3d0.shape

(300, 724, 1024)

In [5]:
import sys
import numpy as np
import os
from wiggler_radiation.Wigrad.wigrad_generator import get_rad_mesh_tuple
en = 100
rad_mesh_tuple = get_rad_mesh_tuple() 
x1d, y1d, l1d = rad_mesh_tuple
nx, ny, nl = [len(v) for v in rad_mesh_tuple]
def get_step(arr):
    return (arr[-1]-arr[0])/(len(arr)-1)
dx, dy, dl = [get_step(v) for v in get_rad_mesh_tuple()]
tot = dx*dy*dl*np.sum(i3d0)
x1_2D, x2_2D = np.meshgrid(x1d, x1d)
x1m2_2d = x1_2D-x2_2D
y1_2D, y2_2D = np.meshgrid(y1d, y1d)
y1m2_2d = y1_2D-y2_2D

In [6]:
x2d, y2d = np.meshgrid(x1d, y1d)

In [11]:
def rotate_slice(slice2d, phi):
    f = RegularGridInterpolator((y1d, x1d), slice2d, bounds_error=False, fill_value=0)
    x2dphi = x2d*np.cos(phi)+y2d*np.sin(phi)
    y2dphi = -x2d*np.sin(phi)+y2d*np.cos(phi)
    x1dphi = x2dphi.ravel()
    y1dphi = y2dphi.ravel()
    pts = np.array([y1dphi, x1dphi]).T
    return f(pts).reshape(ny, nx)

In [12]:
def rotate_i3d(i3d0, phi):
    i3d = np.zeros(shape=i3d0.shape)
    for i in range(i3d0.shape[0]):
        i3d[i] = rotate_slice(i3d0[i,:,:], phi)
    return i3d

In [13]:
i3dphi = rotate_i3d(i3d0, np.pi*5/180)

In [27]:
def get_Mxy(i3d, sx, sy):
    res = 0
    for slice2d, lmda in zip(i3d, l1d):
        Eexpx = np.tensordot(slice2d,
            lmda*np.exp(-(sx*2*np.pi/lmda*x1m2_2d)**2),
                             (1,0))
        Eexpy = np.tensordot(slice2d,
            lmda*np.exp(-(sy*2*np.pi/lmda*y1m2_2d)**2),
                             (0,1))
        res += np.tensordot(Eexpx, Eexpy.T)
    return tot**2/(dx*dy*dx*dy*dl*1/(2*np.sqrt(np.pi)*1e4)*res)
sx, sy = 1000, 80
sz = 40

In [23]:
Mxy0 = get_Mxy(i3d0, sx, sy)

In [28]:
sz*Mxy0

4537179.565875301

In [25]:
Mxyphi = get_Mxy(i3dphi, sx, sy)

In [29]:
sz*Mxyphi

4536980.055318462